# Get Data from SQL
Set the weight of voting

In [85]:
# Modules for string cleanup
import re

import pandas as pd

# for manipulating date data in df
import datetime

# for importing category labelings used for converting into vectors
import json

# model: catboost
import catboost as cb

In [2]:
# Connecting to the SQL server
import psycopg2
from psycopg2.extras import RealDictCursor

In [3]:
%run dbconn.ipynb

In [4]:
conn.autocommit = True

In [5]:
data = pd.DataFrame()

# data for all

In [6]:
# get state from DB and append to data DF
state = []

cur = conn.cursor(cursor_factory=RealDictCursor)
sql = "select state from ml.testcrawl2"
cur.execute(sql)
res = cur.fetchall()

for i in res:
    state.append(i['state'])
    
data['state'] = state

# binnarize the dependent variable
data['state_bin'] = data['state'].apply(lambda x: 0 if x == 'fail' else 1)

# data for NLP

In [7]:
# get blurb from DB and append to data DF
blurb = []

cur = conn.cursor(cursor_factory=RealDictCursor)
sql = "select blurb from ml.testcrawl2"
cur.execute(sql)
res = cur.fetchall()

for i in res:
    blurb.append(i['blurb'])
    
data['blurb'] = blurb

# data for non-NLP

In [57]:
# get category from DB
category = []

cur = conn.cursor(cursor_factory=RealDictCursor)
sql = "select category from ml.testcrawl2"
cur.execute(sql)
res = cur.fetchall()

for i in res:
    category.append(i['category'])
    
data['main_category'] = category

In [58]:
# get currency_type from DB
currency_type = []

cur = conn.cursor(cursor_factory=RealDictCursor)
sql = "select currency_type from ml.testcrawl2"
cur.execute(sql)
res = cur.fetchall()

for i in res:
    currency_type.append(i['currency_type'])
    
data['currency'] = currency_type

In [59]:
# get usd_goal_real from DB
usd_goal_real = []

cur = conn.cursor(cursor_factory=RealDictCursor)
sql = "select usd_goal_real from ml.testcrawl2"
cur.execute(sql)
res = cur.fetchall()

for i in res:
    usd_goal_real.append(i['usd_goal_real'])
    
data['usd_goal_real'] = usd_goal_real

In [60]:
# get usd_pledged_real from DB
usd_pledged_real = []

cur = conn.cursor(cursor_factory=RealDictCursor)
sql = "select usd_pledged_real from ml.testcrawl2"
cur.execute(sql)
res = cur.fetchall()

for i in res:
    usd_pledged_real.append(i['usd_pledged_real'])
    
data['usd_pledged_real'] = usd_pledged_real

In [61]:
# get launched data from DB
launched = []

cur = conn.cursor(cursor_factory=RealDictCursor)
sql = "select launched from ml.testcrawl2"
cur.execute(sql)
res = cur.fetchall()

for i in res:
    launched.append(i['launched'])
    
data['launched'] = launched

In [62]:
# get deadline data from DB
deadline = []

cur = conn.cursor(cursor_factory=RealDictCursor)
sql = "select deadline from ml.testcrawl2"
cur.execute(sql)
res = cur.fetchall()

for i in res:
    deadline.append(i['deadline'])
    
data['deadline'] = deadline

In [63]:
# get country data from DB
country = []

cur = conn.cursor(cursor_factory=RealDictCursor)
sql = "select country from ml.testcrawl2"
cur.execute(sql)
res = cur.fetchall()

for i in res:
    country.append(i['country'])
    
data['country'] = country

In [64]:
projects = data[data['country']!=''].copy()

In [65]:
projects['launched_year'] = projects.apply(lambda x: x['launched'].split(" ")[0].split("-")[0], axis = 1)
projects['launched_month'] = projects.apply(lambda x: x['launched'].split(" ")[0].split("-")[1], axis = 1)
projects['launched_day'] = projects.apply(lambda x: x['launched'].split(" ")[0].split("-")[2], axis = 1)
projects['launched_date'] = projects.apply(lambda x: x['launched'].split(" ")[0].split("-")[1] + x['launched'].split(" ")[0].split("-")[2], axis = 1)

In [66]:
projects['deadline_dt'] = projects.apply(lambda x: datetime.datetime.strptime(x['deadline'], "%Y-%m-%d").date(), axis = 1)
projects['launched_dt'] = projects.apply(lambda x: datetime.datetime.strptime(x['launched'].split(" ")[0], "%Y-%m-%d").date(), axis = 1)

In [67]:
projects['term'] = projects.apply(lambda x: int(str(x['deadline_dt'] - x['launched_dt']).split(" ")[0]), axis = 1)
projects['term_str'] = projects.apply(lambda x: str(x['term']), axis = 1)

In [110]:
# 중도 취소로 인해 term이 왜곡된 row 삭제
projects = projects[lambda x: x['term'] > 1]

In [111]:
# launch 날짜가 1970년으로 되어있는 row는 삭제
projects_new = projects[lambda x: x['term'] < 100]

In [112]:
projects_new = projects_new.reset_index().drop(['index'], axis = 1)

In [113]:
projects_new['term_str'] = projects_new.apply(lambda x: x['term_str'] if len(x['term_str']) != 1
                                             else '0' + x['term_str'], axis = 1)

In [114]:
# 10만원 이하로 펀딩 받는 경우는 지움
projects_new = projects_new[projects_new['usd_goal_real'] > 100]

In [115]:
projects_new = projects_new[projects_new['country'].isin(['US', 'UK', 'Canada', 'Australia', 'Germany', 'France', 'Italy', 'Netherlands', 'Spain', 'Singapore'])]

In [116]:
projects_fin = projects_new[projects_new['currency'].isin(['USD', 'GBP', 'EUR', 'CAD', 'AUD', 'SGD'])]

# Processing the Data into a Trainable data

In [117]:
# Creating the dataset
train_data = projects_fin[['main_category', 'currency', 'country', 'usd_goal_real', 'state_bin', 'launched_year', 'launched_month', 'term']].copy()

In [118]:
train = pd.DataFrame()

In [119]:
# Assigning a categorical type to categorical varibles and vectorizing the categories for simplification
train['main_category'] = train_data['main_category'].astype('category').cat.codes
train['currency'] = train_data['currency'].astype('category').cat.codes
train['country'] = train_data['country'].astype('category').cat.codes

In [120]:
# export labes for vectorizing

main_c = train_data['main_category'].astype('category')
curre_c = train_data['currency'].astype('category')
count_c = train_data['country'].astype('category')
main_d = dict(enumerate(main_c.cat.categories))
curre_d = dict(enumerate(curre_c.cat.categories))
count_d = dict(enumerate(count_c.cat.categories))

main = {y:x for x,y in main_d.items()}
curre = {y:x for x,y in curre_d.items()}
count = {y:x for x,y in count_d.items()}

cat_labels = {'main_category' : main, 'currency' : curre, 'country': count}
cat_json = json.dumps(cat_labels)
f = open("cat_labels.json","w")
f.write(cat_json)
f.close()

In [121]:
train[['usd_goal_real', 'launched_year', 'launched_month', 'term', 'state']] = train_data[['usd_goal_real', 'launched_year', 'launched_month', 'term', 'state_bin']]

In [122]:
train['launched_year'] = (train['launched_year']).astype('int')
train['launched_month'] = (train['launched_month']).astype('int')

In [123]:
cb_clf = cb.CatBoostClassifier(task_type = 'GPU')

In [124]:
# Labelling the categorical features
cat_features= [0,1,2]

# define train set
x = train.iloc[:,:-1]
y = train.iloc[:,-1]

# train model
cb_clf.fit(x, y, cat_features)

Learning rate set to 0.006966
0:	learn: 0.6881329	total: 40.1ms	remaining: 40s
1:	learn: 0.6821029	total: 85.1ms	remaining: 42.5s
2:	learn: 0.6759693	total: 129ms	remaining: 42.9s
3:	learn: 0.6695350	total: 163ms	remaining: 40.6s
4:	learn: 0.6623348	total: 209ms	remaining: 41.6s
5:	learn: 0.6564993	total: 255ms	remaining: 42.2s
6:	learn: 0.6512672	total: 299ms	remaining: 42.4s
7:	learn: 0.6467141	total: 341ms	remaining: 42.2s
8:	learn: 0.6414456	total: 387ms	remaining: 42.6s
9:	learn: 0.6369475	total: 435ms	remaining: 43.1s
10:	learn: 0.6323781	total: 476ms	remaining: 42.8s
11:	learn: 0.6266655	total: 524ms	remaining: 43.2s
12:	learn: 0.6209220	total: 570ms	remaining: 43.3s
13:	learn: 0.6164808	total: 615ms	remaining: 43.3s
14:	learn: 0.6106010	total: 657ms	remaining: 43.1s
15:	learn: 0.6059205	total: 705ms	remaining: 43.4s
16:	learn: 0.6013853	total: 757ms	remaining: 43.8s
17:	learn: 0.5975124	total: 803ms	remaining: 43.8s
18:	learn: 0.5931997	total: 848ms	remaining: 43.8s
19:	learn: 

161:	learn: 0.2518855	total: 7.31s	remaining: 37.8s
162:	learn: 0.2511892	total: 7.36s	remaining: 37.8s
163:	learn: 0.2488240	total: 7.4s	remaining: 37.7s
164:	learn: 0.2482892	total: 7.44s	remaining: 37.6s
165:	learn: 0.2471388	total: 7.46s	remaining: 37.5s
166:	learn: 0.2462534	total: 7.51s	remaining: 37.5s
167:	learn: 0.2440009	total: 7.56s	remaining: 37.5s
168:	learn: 0.2431099	total: 7.61s	remaining: 37.4s
169:	learn: 0.2420128	total: 7.66s	remaining: 37.4s
170:	learn: 0.2407302	total: 7.7s	remaining: 37.3s
171:	learn: 0.2400057	total: 7.75s	remaining: 37.3s
172:	learn: 0.2390609	total: 7.79s	remaining: 37.3s
173:	learn: 0.2383914	total: 7.83s	remaining: 37.2s
174:	learn: 0.2377736	total: 7.88s	remaining: 37.2s
175:	learn: 0.2365015	total: 7.93s	remaining: 37.1s
176:	learn: 0.2357305	total: 7.97s	remaining: 37.1s
177:	learn: 0.2349813	total: 8.02s	remaining: 37s
178:	learn: 0.2340117	total: 8.06s	remaining: 37s
179:	learn: 0.2333126	total: 8.1s	remaining: 36.9s
180:	learn: 0.23262

321:	learn: 0.1373519	total: 14.6s	remaining: 30.7s
322:	learn: 0.1369916	total: 14.6s	remaining: 30.7s
323:	learn: 0.1367798	total: 14.7s	remaining: 30.6s
324:	learn: 0.1360855	total: 14.7s	remaining: 30.6s
325:	learn: 0.1358369	total: 14.8s	remaining: 30.5s
326:	learn: 0.1356155	total: 14.8s	remaining: 30.5s
327:	learn: 0.1350119	total: 14.8s	remaining: 30.4s
328:	learn: 0.1347858	total: 14.9s	remaining: 30.4s
329:	learn: 0.1345745	total: 14.9s	remaining: 30.3s
330:	learn: 0.1339947	total: 15s	remaining: 30.3s
331:	learn: 0.1338114	total: 15s	remaining: 30.2s
332:	learn: 0.1333043	total: 15.1s	remaining: 30.2s
333:	learn: 0.1330753	total: 15.1s	remaining: 30.1s
334:	learn: 0.1326999	total: 15.2s	remaining: 30.1s
335:	learn: 0.1323558	total: 15.2s	remaining: 30s
336:	learn: 0.1320632	total: 15.2s	remaining: 30s
337:	learn: 0.1318738	total: 15.3s	remaining: 30s
338:	learn: 0.1315547	total: 15.3s	remaining: 29.9s
339:	learn: 0.1313595	total: 15.4s	remaining: 29.9s
340:	learn: 0.1309829	

483:	learn: 0.0869361	total: 21.9s	remaining: 23.4s
484:	learn: 0.0867637	total: 22s	remaining: 23.3s
485:	learn: 0.0866415	total: 22s	remaining: 23.3s
486:	learn: 0.0863333	total: 22.1s	remaining: 23.3s
487:	learn: 0.0860439	total: 22.1s	remaining: 23.2s
488:	learn: 0.0858215	total: 22.2s	remaining: 23.2s
489:	learn: 0.0856615	total: 22.2s	remaining: 23.1s
490:	learn: 0.0855983	total: 22.2s	remaining: 23.1s
491:	learn: 0.0850557	total: 22.3s	remaining: 23s
492:	learn: 0.0847969	total: 22.3s	remaining: 23s
493:	learn: 0.0845588	total: 22.4s	remaining: 22.9s
494:	learn: 0.0843152	total: 22.4s	remaining: 22.9s
495:	learn: 0.0842309	total: 22.5s	remaining: 22.8s
496:	learn: 0.0840383	total: 22.5s	remaining: 22.8s
497:	learn: 0.0838913	total: 22.6s	remaining: 22.7s
498:	learn: 0.0836462	total: 22.6s	remaining: 22.7s
499:	learn: 0.0834107	total: 22.6s	remaining: 22.6s
500:	learn: 0.0831630	total: 22.7s	remaining: 22.6s
501:	learn: 0.0829879	total: 22.7s	remaining: 22.5s
502:	learn: 0.082853

645:	learn: 0.0574475	total: 29.4s	remaining: 16.1s
646:	learn: 0.0573161	total: 29.4s	remaining: 16.1s
647:	learn: 0.0572003	total: 29.5s	remaining: 16s
648:	learn: 0.0570907	total: 29.6s	remaining: 16s
649:	learn: 0.0570070	total: 29.6s	remaining: 15.9s
650:	learn: 0.0568479	total: 29.7s	remaining: 15.9s
651:	learn: 0.0567220	total: 29.7s	remaining: 15.9s
652:	learn: 0.0566058	total: 29.8s	remaining: 15.8s
653:	learn: 0.0563538	total: 29.8s	remaining: 15.8s
654:	learn: 0.0561928	total: 29.8s	remaining: 15.7s
655:	learn: 0.0561085	total: 29.9s	remaining: 15.7s
656:	learn: 0.0558241	total: 29.9s	remaining: 15.6s
657:	learn: 0.0555909	total: 30s	remaining: 15.6s
658:	learn: 0.0553267	total: 30s	remaining: 15.5s
659:	learn: 0.0549877	total: 30.1s	remaining: 15.5s
660:	learn: 0.0548525	total: 30.1s	remaining: 15.4s
661:	learn: 0.0547194	total: 30.2s	remaining: 15.4s
662:	learn: 0.0546080	total: 30.2s	remaining: 15.4s
663:	learn: 0.0545676	total: 30.3s	remaining: 15.3s
664:	learn: 0.054415

806:	learn: 0.0409021	total: 36.9s	remaining: 8.83s
807:	learn: 0.0408625	total: 37s	remaining: 8.79s
808:	learn: 0.0408387	total: 37s	remaining: 8.74s
809:	learn: 0.0406776	total: 37.1s	remaining: 8.7s
810:	learn: 0.0406372	total: 37.1s	remaining: 8.65s
811:	learn: 0.0405489	total: 37.2s	remaining: 8.61s
812:	learn: 0.0404925	total: 37.2s	remaining: 8.56s
813:	learn: 0.0404310	total: 37.3s	remaining: 8.52s
814:	learn: 0.0402859	total: 37.3s	remaining: 8.47s
815:	learn: 0.0402598	total: 37.4s	remaining: 8.43s
816:	learn: 0.0401946	total: 37.4s	remaining: 8.38s
817:	learn: 0.0401469	total: 37.5s	remaining: 8.34s
818:	learn: 0.0400086	total: 37.5s	remaining: 8.29s
819:	learn: 0.0398082	total: 37.6s	remaining: 8.24s
820:	learn: 0.0397774	total: 37.6s	remaining: 8.2s
821:	learn: 0.0396993	total: 37.7s	remaining: 8.15s
822:	learn: 0.0395889	total: 37.7s	remaining: 8.11s
823:	learn: 0.0395476	total: 37.8s	remaining: 8.06s
824:	learn: 0.0394751	total: 37.8s	remaining: 8.02s
825:	learn: 0.0394

966:	learn: 0.0313206	total: 44.7s	remaining: 1.52s
967:	learn: 0.0312452	total: 44.7s	remaining: 1.48s
968:	learn: 0.0312054	total: 44.8s	remaining: 1.43s
969:	learn: 0.0311855	total: 44.8s	remaining: 1.39s
970:	learn: 0.0311079	total: 44.9s	remaining: 1.34s
971:	learn: 0.0310734	total: 44.9s	remaining: 1.29s
972:	learn: 0.0310488	total: 45s	remaining: 1.25s
973:	learn: 0.0310098	total: 45s	remaining: 1.2s
974:	learn: 0.0309358	total: 45.1s	remaining: 1.16s
975:	learn: 0.0309080	total: 45.1s	remaining: 1.11s
976:	learn: 0.0308660	total: 45.2s	remaining: 1.06s
977:	learn: 0.0307384	total: 45.2s	remaining: 1.02s
978:	learn: 0.0306544	total: 45.2s	remaining: 971ms
979:	learn: 0.0305745	total: 45.3s	remaining: 924ms
980:	learn: 0.0305607	total: 45.3s	remaining: 878ms
981:	learn: 0.0305068	total: 45.4s	remaining: 832ms
982:	learn: 0.0304921	total: 45.4s	remaining: 786ms
983:	learn: 0.0304613	total: 45.5s	remaining: 739ms
984:	learn: 0.0304094	total: 45.5s	remaining: 693ms
985:	learn: 0.030

In [125]:
import pickle

filename = 'Non_nlp_model' # name to store model
pickle.dump(cb_clf, open(filename, 'wb')) # pickling

In [126]:
import re

def clean(text):
    text = str(text)
    text = re.findall(r'\w+', text)
    return ' '.join(text)
data['blurb'] = data['blurb'].apply(lambda x: clean(x))

In [127]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer().fit(data['blurb'])
x = vectorizer.transform(data['blurb'])

In [135]:
filename = 'vectorizer.sav'
pickle.dump(vectorizer, open(filename, 'wb'))

In [128]:
from sklearn.linear_model import LogisticRegression

sgd = LogisticRegression()

In [129]:
sgd.fit(x, data['state_bin'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [130]:
filename = 'NLP_model.sav'
pickle.dump(sgd, open(filename, 'wb'))